# CRUD básico com sqlalchemy

## Carregando variáveis de ambiente

In [1]:
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

user_db = os.getenv("POSTGRES_USER")
passwd_db = os.getenv("POSTGRES_PASSWORD")
db = os.getenv("POSTGRES_DB")
host_db = os.getenv("POSTGRES_HOST")
port_db = os.getenv("POSTGRES_PORT")

## Criando engine de conexão com o banco de dados

In [2]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, create_engine

conn = f"postgresql://{user_db}:{passwd_db}@{host_db}:{port_db}/{db}"
engine = create_engine(conn, echo=True)

In [3]:
Base = declarative_base()

In [4]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    username = Column(String)
    password = Column(String)

In [5]:
Base.metadata.create_all(engine)

2024-02-22 14:51:38,641 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-02-22 14:51:38,642 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-22 14:51:38,643 INFO sqlalchemy.engine.Engine select current_schema()
2024-02-22 14:51:38,644 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-22 14:51:38,645 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-02-22 14:51:38,645 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-22 14:51:38,646 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-22 14:51:38,648 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

## Inserir um usuário no banco de dados

In [8]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

user = User(username='admin', password='admin')
session.add(user)
session.commit()

2024-02-22 14:56:38,588 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-22 14:56:38,590 INFO sqlalchemy.engine.Engine INSERT INTO users (username, password) VALUES (%(username)s, %(password)s) RETURNING users.id
2024-02-22 14:56:38,591 INFO sqlalchemy.engine.Engine [generated in 0.00049s] {'username': 'admin', 'password': 'admin'}
2024-02-22 14:56:38,592 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
# Inserir vários usuários

users = [
    User(username='user1', password='user1'),
    User(username='user2', password='user2'),
    User(username='user3', password='user3'),
    User(username='user4', password='user4'),
    User(username='user5', password='user5'),
]

session.add_all(users)
session.commit()

2024-02-22 14:59:05,916 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-22 14:59:05,918 INFO sqlalchemy.engine.Engine INSERT INTO users (username, password) SELECT p0::VARCHAR, p1::VARCHAR FROM (VALUES (%(username__0)s, %(password__0)s, 0), (%(username__1)s, %(password__1)s, 1), (%(username__2)s, %(password__2)s, 2), (%(username__3)s, %(password__3)s, 3), (%(username__4)s, %(password__4)s, 4)) AS imp_sen(p0, p1, sen_counter) ORDER BY sen_counter RETURNING users.id, users.id AS id__1
2024-02-22 14:59:05,919 INFO sqlalchemy.engine.Engine [generated in 0.00013s (insertmanyvalues) 1/1 (ordered)] {'username__0': 'user1', 'password__0': 'user1', 'username__1': 'user2', 'password__1': 'user2', 'username__2': 'user3', 'password__2': 'user3', 'username__3': 'user4', 'password__3': 'user4', 'username__4': 'user5', 'password__4': 'user5'}
2024-02-22 14:59:05,921 INFO sqlalchemy.engine.Engine COMMIT


## Consultar na tabela

In [13]:
# listar todos

query_users = session.query(User).all()
for user in query_users:
    print(user.username, user.password)

2024-02-22 15:02:58,620 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password 
FROM users
2024-02-22 15:02:58,621 INFO sqlalchemy.engine.Engine [cached since 42.54s ago] {}
admin admin
user1 user1
user2 user2
user3 user3
user4 user4
user5 user5


In [17]:
# filtrando por username

query_users = session.query(User).filter(User.username == 'admin').first()
print(query_users.username, query_users.password)

2024-02-22 15:04:04,487 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password 
FROM users 
WHERE users.username = %(username_1)s 
 LIMIT %(param_1)s
2024-02-22 15:04:04,488 INFO sqlalchemy.engine.Engine [cached since 45.02s ago] {'username_1': 'admin', 'param_1': 1}
admin admin


## Atualizar um usuário

In [19]:
user.username

# Atualizando

query_users = session.query(User).filter(User.username == 'admin').first()
query_users.password = 'newpassword'
session.commit()

2024-02-22 15:07:25,851 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password 
FROM users 
WHERE users.username = %(username_1)s 
 LIMIT %(param_1)s
2024-02-22 15:07:25,852 INFO sqlalchemy.engine.Engine [cached since 246.4s ago] {'username_1': 'admin', 'param_1': 1}
2024-02-22 15:07:25,854 INFO sqlalchemy.engine.Engine UPDATE users SET password=%(password)s WHERE users.id = %(users_id)s
2024-02-22 15:07:25,854 INFO sqlalchemy.engine.Engine [generated in 0.00042s] {'password': 'newpassword', 'users_id': 1}
2024-02-22 15:07:25,855 INFO sqlalchemy.engine.Engine COMMIT


In [22]:
query_users = session.query(User).filter(User.username == 'admin').first()
print(query_users.username, query_users.password)

2024-02-22 15:08:04,601 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password 
FROM users 
WHERE users.username = %(username_1)s 
 LIMIT %(param_1)s
2024-02-22 15:08:04,601 INFO sqlalchemy.engine.Engine [cached since 285.1s ago] {'username_1': 'admin', 'param_1': 1}
admin newpassword


## Apagar um usuário

In [23]:
# Apagar o usuário 3

query_users = session.query(User).filter(User.username == 'user3').first()
session.delete(query_users)
session.commit()

2024-02-22 15:09:25,752 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password 
FROM users 
WHERE users.username = %(username_1)s 
 LIMIT %(param_1)s
2024-02-22 15:09:25,753 INFO sqlalchemy.engine.Engine [cached since 366.3s ago] {'username_1': 'user3', 'param_1': 1}
2024-02-22 15:09:25,754 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = %(id)s
2024-02-22 15:09:25,754 INFO sqlalchemy.engine.Engine [generated in 0.00035s] {'id': 4}
2024-02-22 15:09:25,755 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
query_users = session.query(User).all()
for user in query_users:
    print(user.username, user.password)

2024-02-22 15:09:37,911 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-22 15:09:37,912 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.password AS users_password 
FROM users
2024-02-22 15:09:37,912 INFO sqlalchemy.engine.Engine [cached since 441.8s ago] {}
user1 user1
user2 user2
user4 user4
user5 user5
admin newpassword
